## Basic

In [1]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)


## Dataset

In [2]:
from datasets import get_dataset_config_names

HF_DATASET_REPO_NAME = "liamsbhoo/GiftEvalPretrain"
# get_dataset_config_names(HF_DATASET_REPO_NAME)

2025-04-28 22:24:03 - INFO - PyTorch version 2.5.1 available.


In [3]:
from tabpfn_time_series.experimental.finetuning.dataset import TabPFNTimeSeriesPretrainDataset, TimeSeriesPretrainConfig

pretrain_config = TimeSeriesPretrainConfig()
pretrain_config.max_context_length = 500

dataset = TabPFNTimeSeriesPretrainDataset(
    dataset_repo_name="liamsbhoo/GiftEvalPretrain",
    dataset_names=[
        "borealis",
        "wind_power",
    ],
)


2025-04-28 22:24:06 - INFO - Loading datasets: ['borealis', 'wind_power'] from liamsbhoo/GiftEvalPretrain


Resolving data files:   0%|          | 0/6376 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/6376 [00:00<?, ?it/s]

2025-04-28 22:24:34 - INFO - Loaded 2 datasets
2025-04-28 22:24:34 - INFO -   - borealis: 15 samples
2025-04-28 22:24:34 - INFO -   - wind_power: 1 samples


In [4]:
# Load the autoreload extension to automatically reload modules when they change
%load_ext autoreload
%autoreload 2

# This ensures that changes to imported modules are reloaded without restarting the kernel
# Particularly useful during development when modifying the dataset.py or other modules


In [5]:
%run -m tabpfn_time_series.experimental.finetuning.dataset
from tabpfn_time_series.experimental.finetuning.dataset import load_all_ts_datasets

all_X, all_y = load_all_ts_datasets(dataset)

/Users/shibinhoo/miniconda3/envs/tabpfn-ts-ft/lib/python3.10/runpy.py:126: RuntimeWarning: 'tabpfn_time_series.experimental.finetuning.dataset' found in sys.modules after import of package 'tabpfn_time_series.experimental.finetuning', but prior to execution of 'tabpfn_time_series.experimental.finetuning.dataset'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))


In [6]:
len(all_X)

16

In [33]:
import torch
from tabpfn import TabPFNRegressor

regressor_args = dict(
    ignore_pretraining_limits=True,
    device="cpu",
    n_estimators=8,
    random_state=0,
    inference_precision=torch.float32,
)
    
reg = TabPFNRegressor(
    **regressor_args,
    differentiable_input=False,
)



In [34]:
from functools import partial
from sklearn.model_selection import train_test_split

# splitfn = partial(train_test_split, test_size=0.2, random_state=42)
def ts_splitfn(data, **kwargs):
    prediction_length = 7
    print("data type", type(data))
    # Handle pandas DataFrame/Series properly
    if isinstance(data, pd.DataFrame) or isinstance(data, pd.Series):
        train_data = data.iloc[:-prediction_length]
        test_data = data.iloc[-prediction_length:]
    else:
        # For numpy arrays or other array-like objects
        train_data = data[:-prediction_length]
        test_data = data[-prediction_length:]
    return train_data, test_data

splitfn = partial(train_test_split, test_size=0.2, random_state=42)

datasets_collection = reg.get_preprocessed_datasets(all_X, all_y, splitfn, max_data_size=1000)
# datasets_collection_test = reg.get_preprocessed_datasets(X_tests[0], y_tests[0], splitfn, max_data_size=150)

/Users/shibinhoo/Documents/freiburg-cs/tabpfn-time-series/tabpfn_time_series/experimental/finetuning/tabpfn_private/src/tabpfn/regressor.py:533: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  check_cpu_warning(self.device, X_raw)
/Users/shibinhoo/Documents/freiburg-cs/tabpfn-time-series/tabpfn_time_series/experimental/finetuning/tabpfn_private/src/tabpfn/regressor.py:533: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  check_cpu_warning(self.device, X_raw)
/Users/shibinhoo/Documents/freiburg-cs/tabpfn-time-series/tabpfn_time_series/experimental/finetuning/tabpfn_private/src/tabpfn/regressor.py:533: UserWarning: Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client
  check_cpu

In [39]:
from tqdm import tqdm

from torch.utils.data import DataLoader
from tabpfn.utils import collate_for_tabpfn_dataset

EPOCHS = 2

train_dl = DataLoader(datasets_collection, batch_size=1, collate_fn=collate_for_tabpfn_dataset)

for epoch in range(EPOCHS):
    batch_iterator = tqdm(train_dl, desc=f"Epoch {epoch+1}/{EPOCHS}")
    for data_batch in batch_iterator:
        print("helo")
        (X_trains_preprocessed, X_tests_preprocessed, y_trains_preprocessed,
         y_test_standardized, cat_ixs, confs, renormalized_criterion, 
         batch_x_test_raw, batch_y_test_raw) = data_batch

        reg.fit_from_preprocessed(X_trains_preprocessed, y_trains_preprocessed, cat_ixs, confs)

        averaged_pred_logits , _, _= reg.forward(X_tests_preprocessed) # [BatchSize, N_test, NumBars]  

        

Epoch 1/2:   0%|          | 0/79 [00:00<?, ?it/s]

helo


Epoch 1/2:   0%|          | 0/79 [00:08<?, ?it/s]


KeyboardInterrupt: 